In [1]:
from dotenv import load_dotenv
import os
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from pinecone.core.grpc.protos.vector_service_pb2 import UpsertResponse
from pinecone.grpc.future import PineconeGrpcFuture
from rapid_law.components.cohere_utils import cohere_embed

/Users/snehvora/Desktop/rapid_law/.venv/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()

# Access environment variables
pinecone_api_key = os.getenv('PINECONE_API_KEY')

In [3]:
pc = Pinecone(api_key=pinecone_api_key)

In [4]:
def create_index(
    index_name : str,
    dimension : int,
    cloud : str = 'aws',
    region : str = 'us-east-1',
    metric : str = "cosine"
):
    try:
        if index_name not in pc.list_indexes().names():
            pc.create_index(
                name=index_name,
                dimension=dimension,
                metric=metric,
                spec=ServerlessSpec(
                    cloud=cloud, 
                    region=region
                ) 
            ) 
    except Exception as e:
        print(f"An error occurred in create_index function: {e}")
    

In [5]:
create_index("rapid-laws",1024)

In [6]:
def pinecone_upsert(
    index_name : str,
    title : str,
    text : str,
    metadata : dict,
    namespace : str = "ns1"
)->(UpsertResponse | PineconeGrpcFuture):

    index = pc.Index(index_name)

    value = cohere_embed(text=text,)
    
    index.upsert(
        vectors=[
            {
                "id": title, 
                "values": value, 
                "metadata": metadata
            }
        ],
        namespace=namespace
    )
    return (UpsertResponse | PineconeGrpcFuture)

In [7]:
x = pinecone_upsert("rapid-laws","for living Document","What do you do for living?",{"text" : "What do you do for living?"})
print(x)

vector_service_pb2.UpsertResponse | pinecone.grpc.future.PineconeGrpcFuture


In [4]:
def pinecone_query(
    index_name : str,
    query : str,
    namespace : str = "ns1",
    top_k : int = 3,
    include_values : bool = False,
    include_metadata : bool = True
):
    index = pc.Index(index_name)
    value = cohere_embed(text=query)

    response = index.query(
        namespace=namespace,
        vector=value,
        top_k=top_k,
        include_values=include_values,
        include_metadata=include_metadata
    )

    return response

In [8]:
x = pinecone_query("rapid-laws", "exempt from such registration under such section pursuant to an order of the Securities and Exchange", include_values=False)
print(x)

{'matches': [], 'namespace': 'ns1', 'usage': {'read_units': 1}}


In [10]:
x = pinecone_upsert("rapid-laws","Asking Name","Who are You",{"text" : "Who are You"})
print(x)

vector_service_pb2.UpsertResponse | pinecone.grpc.future.PineconeGrpcFuture


In [5]:
pinecone_query("rapid-laws-768", " exempt from such registration under such section pursuant to an order of the Securities and Exchange",top_k=10)

{'matches': [{'id': '302',
              'metadata': {'chapter': 'Chapter 5—Creditors, The Debtor, And '
                                      'The Estate (§§ 501 – 562)',
                           'section': '§\u202f524. Effect of discharge',
                           'section_content': '. It is not discharged in your '
                                              'bankruptcy case. That means '
                                              'that if you default on your '
                                              'reaffirmed debt after your '
                                              'bankruptcy case is over, your '
                                              'creditor may be able to take '
                                              'your property or your wages. '
                                              'Otherwise, your obligations '
                                              'will be determined by the '
                                              'reaffirmati

In [17]:
pinecone_query("rapid-laws-768", " In this title the following definitions shall apply: (1) The term “accountant” means accountant authorized under")

{'matches': [{'id': '316',
              'metadata': {'chapter': 'Chapter 5—Creditors, The Debtor, And '
                                      'The Estate (§§ 501 – 562)',
                           'section': '§\u202f527. Disclosures',
                           'section_content': ' (a) A debt relief agency '
                                              'providing bankruptcy assistance '
                                              'to an assisted person shall '
                                              'provide— (1)  the written '
                                              'notice required under section '
                                              '342(b)(1); and  (2) to the '
                                              'extent not covered in the '
                                              'written notice described in '
                                              'paragraph (1), and not later '
                                              'than 3 business days af